In [4]:
from urllib.error import URLError
from urllib.request import ProxyHandler,build_opener

In [14]:
proxy = '110.52.235.85:9999'
proxy_handler = ProxyHandler({
    'http':'http://'+proxy,
    'https':'https://'+proxy
})
opener = build_opener(proxy_handler)
response = opener.open('http://httpbin.org/get')
print(response.read().decode('utf-8'))

{
  "args": {}, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Cache-Control": "max-age=259200", 
    "Connection": "close", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.7"
  }, 
  "origin": "110.52.235.85", 
  "url": "http://httpbin.org/get"
}



In [6]:
from urllib.request import urlopen

In [9]:
doc = urlopen("http://httpbin.org/get").read()
print(doc)

b'{\n  "args": {}, \n  "headers": {\n    "Accept-Encoding": "identity", \n    "Connection": "close", \n    "Host": "httpbin.org", \n    "User-Agent": "Python-urllib/3.7"\n  }, \n  "origin": "112.5.203.142", \n  "url": "http://httpbin.org/get"\n}\n'


In [10]:
opener.open

<bound method OpenerDirector.open of <urllib.request.OpenerDirector object at 0x7f11d87ed128>>

 # Socket

因为jupyter notebook只支持一个进程，所以我们在这里新建了另一个文件来实现客户端
## 服务端(TCP编程)
在使用WireShark进行抓包的时候，因为wireshark本身不支持本地回炉抓包，要设置本地回路抓包才可以看到三次握手的结果。
> Capture-> Option->Manage Interfaces->LoopBook:IO

Go!

In [ ]:
import socket
import threading

# AF_INET: 基于 IPV4 的网络通信 SOCK_STREAM: 基于 TCP 的流式 socket 通信
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# 将套接字绑定到地址
s.bind(('127.0.0.1', 8898))
# 监听TCP传入连接
s.listen(5)
def handle_tcp(sock, addr):
    print("new connection from %s:%s" % addr)
    sock.send(b'Welcome!')

    while True:
        data = sock.recv(1024)
        if not data:
            break
        sock.send(b'Hello, %s!' % data)
    sock.close()


while True:
    sock, addr = s.accept()
    t = threading.Thread(target=handle_tcp, args=(sock, addr))
    t.start()

new connection from 127.0.0.1:42282
new connection from 127.0.0.1:42288


## UDP编程（服务端）

In [2]:
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

s.bind(('127.0.0.1',8898))

while True:
    data, addr = s.recvfrom(1024)
    print('Received from %s,%s'%addr)
    s.sendto(b'Hello, %s'%data,addr)

Received from 127.0.0.1,35198
Received from 127.0.0.1,57098
Received from 127.0.0.1,47767
Received from 127.0.0.1,35973
Received from 127.0.0.1,45160


KeyboardInterrupt: 

## socks5

In [5]:
import logging
import socket
import struct
import select
import threading


def send_data(sock, data):
    print(data)
    bytes_sent = 0
    while True:
        r = sock.send(data[bytes_sent:])
        if r < 0:
            return r
        bytes_sent += r
        if bytes_sent == len(data):
            return bytes_sent


def handle_tcp(sock, remote):
    # 处理 client socket 和 remote socket 的数据流
    try:
        fdset = [sock, remote]
        while True:
            # 用 IO 多路复用 select 监听套接字是否有数据流
            r, w, e = select.select(fdset, [], [])
            if sock in r:
                data = sock.recv(4096)
                if len(data) <= 0:
                    break
                result = send_data(remote, data)
                if result < len(data):
                    raise Exception('failed to send all data')

            if remote in r:
                data = remote.recv(4096)
                if len(data) <= 0:
                    break
                result = send_data(sock, data)
                if result < len(data):
                    raise Exception('failed to send all data')
    except Exception as e:
        raise(e)
    finally:
        sock.close()
        remote.close()


def handle_con(sock, addr):
    # 接受客户端来的请求，socks5 的 认证和连接过程

    sock.recv(256)
    # 无需进一步认证信息
    sock.send(b"\x05\x00")
    data = sock.recv(4) or '\x00' * 4
    # CMD 为 0x01 也就是 CONNECT 继续
    mode = data[1]
    if mode != 1:
        return
    # DST.ADDR 有三种形式，分别做判断
    addr_type = data[3]
    if addr_type == 1:
        addr_ip = sock.recv(4)
        remote_addr = socket.inet_ntoa(addr_ip)
    elif addr_type == 3:
        addr_len = int.from_bytes(sock.recv(1), byteorder='big')
        remote_addr = sock.recv(addr_len)
    elif addr_type == 4:
        addr_ip = sock.recv(16)
        remote_addr = socket.inet_ntop(socket.AF_INET6, addr_ip)
    else:
        return
    # DST.PORT
    remote_addr_port = struct.unpack('>H', sock.recv(2))

    # 返回给客户端 success
    reply = b"\x05\x00\x00\x01"
    reply += socket.inet_aton('0.0.0.0') + struct.pack(">H", 8888)
    sock.send(reply)

    # 拿到 remote address 的信息后，建立连接
    try:
        remote = socket.create_connection((remote_addr, remote_addr_port[0]))
        logging.info('connecting %s:%d' % (remote_addr, remote_addr_port[0]))
    except socket.error as e:
        logging.error(e)
        return

    handle_tcp(sock, remote)


def main():
    socketServer = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    socketServer.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

    socketServer.bind(('', 8898))
    socketServer.listen(5)

    try:
        while True:
            sock, addr = socketServer.accept()
            t = threading.Thread(target=handle_con, args=(sock, addr))
            t.start()
    except socket.error as e:
        logging.error(e)
    except KeyboardInterrupt:
        socketServer.close()


if __name__ == '__main__':
    main()

ERROR:root:[Errno -2] Name or service not known
ERROR:root:[Errno -2] Name or service not known
ERROR:root:[Errno -2] Name or service not known


b'GET / HTTP/1.1 \r\n\r\n'
b'HTTP/1.1 400 Bad Request\r\nServer: WeiBo\r\nDate: Sun, 20 Jan 2019 07:17:33 GMT\r\nContent-Type: text/html\r\nContent-Length: 578\r\nConnection: close\r\n\r\n<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\r\n<html>\r\n<head><title>400 Bad Request</title></head>\r\n<body bgcolor="white">\r\n<h1>400 Bad Request</h1>\r\n<p>Your browser sent a request that this server could not understand. Sorry for the inconvenience.<br/>\r\nPlease report this message and include the following information to us.<br/>\r\nThank you very much!</p>\r\n<table>\r\n<tr>\r\n<td>URL:</td>\r\n<td>http:///</td>\r\n</tr>\r\n<tr>\r\n<td>Server:</td>\r\n<td>venus245</td>\r\n</tr>\r\n<tr>\r\n<td>Date:</td>\r\n<td>2019/01/20 15:17:33</td>\r\n</tr>\r\n</table>\r\n<hr/>Powered by WeiBo</body>\r\n</html>\r\n'
b'GET / HTTP/1.1 \r\n\r\n'
b'HTTP/1.1 400 Bad Request\r\nServer: WeiBo\r\nDate: Sun, 20 Jan 2019 07:18:00 GMT\r\nContent-Type: text/html\r\nContent-Length: 578\r\nConnection: close\r\n